In [1]:
import langchain

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the environment variable
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key not found in environment variables")


In [2]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("documents/BME_TVSZ_2016_elfogadott_mod_20220928_T.pdf")
pages = loader.load_and_split()

In [3]:
pages[0]

Document(page_content='A Szenátus  X./10./2015-2016. (201 6. VII. 11.)  \nszámú határozata  \n \nA BME  TANULMÁNYI ÉS  VIZSGA SZABÁLYZAT ÁRÓL  \n \n \nA módosításáról szóló  \nII./3./2016 -2017. , VII./4./2016 -2017. , VIII./4./2016 -2017 ., IV./4./2017 -2018. , X./5./2017 -2018 ., \nIX./2. /2018 -2019. , V./3./2019 -2020. , II./1./2020 -2021. , V./1./2020 -2021. ,X./3./2020 -2021. ,  \nV./2./2021 -2022 ., X./2./2021 -2022., XI./2./2021 -2022. , XII./1./2021 -2022. , I./2./2022 -2023. \nszámú Szenátusi határozat tal egybeszerkesztett változat . \n \n \n \n \nHatályos 2016. szeptember 1. napjától  \n \n \nRektori Kabinet  \nOktatási Igazgatóság', metadata={'source': 'documents/BME_TVSZ_2016_elfogadott_mod_20220928_T.pdf', 'page': 0})

In [5]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [7]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())

In [28]:
question = "Mi a kritériuma a kitüntetéses oklevélnek?"
docs = faiss_index.similarity_search(question, k=2)

for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)
    print("__________________________________________________-")
context = docs[0].page_content + docs[1].page_content

112: 113 meghatározott tartalommal – oklevelet ad ki. Az oklevél kiállítására kizárólag a 
KTH jogosult . 
(1a) 355Az Egyet em az oklevél kiadását mindaddig megtagadja, amíg a hallgató a fel-
vételi eljárásban előírt okiratbemutatási kötelezettségének nem tesz eleget  
 Nem magyar nyelven folyó képzés esetén magyar nyelven és a képzés nyelvén 
kell kiadni az oklevelet.  
 Sikeres záróvi zsga alapján, ha a hallgató az oklevél kiadásához előírt minden fel-
tételnek megfelel, az adatkezelő az oklevél kiállításáról szóló – a kormány rende-
letében meghatározott tartalmú – igazolást ad ki.  
 Ha az oklevél kiadására azért nincs lehetőség, mert a nyelvvizsg át igazoló iratot  a 
hallgató nem mutatta be, a szakot (képzést) gondozó kar  – a hallgató kérésére – 
igazolást állít ki, amely végzettséget és szakképzettséget nem igazol, de tanúsítja 
a záróvizsg a eredményes letételét.  
 Az oklevelet a dékán vagy a z általa kijelölt dékánhelyettes írja alá.  
160. §  [Oklevéleredmény ] 
 Az oklevél

In [19]:
question

'Mi a kritériuma a kitüntetéses oklevélnek?'

In [21]:
len(context)

2705

In [29]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

template = f"""Egy segítőkész aszisztens vagy aki magyar diákoknak segít. A kérdésekre nem tudod a választ valószínűleg,
de emiatt megkapod a releváns információkat amiket felhasználhatsz a válaszadáshoz. Lehetséges, 
hogy a megadott információ tartalmaz nem releváns információt is. Az információ: 
{context}"""

chat = ChatOpenAI()
result = chat([SystemMessage(content=template),HumanMessage(content=question)])

In [35]:
print(result.content)

Az oklevél minősítése kitüntetéses, ha a vizsgázó teljesíti az alábbi feltételeket:

- A záróvizsga minden tantárgyából jeles eredményt ér el.
- A diplomamunkájának vagy szakdolgozatának a záróvizsgán megállapított érdemjegye jeles.
- Amennyiben van, akkor valamennyi, az oklevél megszerzéséhez szükséges szigorlatának osztályzata jeles.
- A teljes tanulmányi időszakra vonatkozó halmozott súlyozott tanulmányi átlaga legalább 4,25.
